In [ ]:
import pandas as pd
import numpy as np
import scipy.signal
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import itertools
import math
import statistics as sts
import gc as gc
import sys
from sklearn import preprocessing


## Aprovechamos los .hdf generados a partir de los .mat en el tp anterior.
path = "../../tp2/datos/"
load_path = path + "/{}.hdf"

##Electrodos de interes
electrodos = [8, 44, 80, 131, 185]

# Cantidad de pacientes S y P
N_P = 2
N_S = 2

<h1>Script Generador de Features</h1>
<blockquote>
    <p>Este script genera los features que luego utilizaremos en el análisis univariado y multivariado.</p>
    <p>Estas son:
    <ul>
        <li>Potencia para cada banda de frecuencia (Delta, Theta, Alpha, Beta y Gamma)</li>
        <li>Potencia normalizada para las mismas bandas de frecuencia.</li>
        <li>Una medida de información intra-electrodo</li>
        <li>Una medida de información inter-electrodo</li>
    </ul>
    </p>
    <p>Cada marcador se computa por cada epochs y luego se toma la media y el desvío estándard entre los valores de cada epochs. En el caso de los features espectrales, para el cómputo de un epoch primero se promedian las frecuencias entre los mismos electrodos utilizados en el TP2.</p>
    <p>Como medida intra-electrodo se decide utilizar la fuente de información modelada en el TP2 considerando sólo al electrodo 8. Se decide esto porque en los resultados observados presentó una mejor diferencia en los rangos de entropía para entre los pacientes P y S.</p>
    <p>La medida inter-electrodo es la entropía de la fuente modelada con un alfabeto general para todos los electrodos (lo mismo que se utilizó en el TP2).</p>
</blockquote>

<h4>Construyendo alfabetos</h4>
<blockquote>
<p>Primero vamos a generar al igual que en el TP anterior los alfabetos que se utilzarán luego en las medidas inter/intra-electrodo.</p>
</blockquote>

In [ ]:
pacientes_P = []
pacientes_S = []
for load_name, N, dest, offset in [("P", N_P, pacientes_P, 0), ("S", N_S, pacientes_S, 10)]:
    for i in range(1, 1 + N):
        paciente = load_name + "{:02d}".format(i)
        df_ = pd.read_hdf(load_path.format(paciente))
        df_ = df_.loc[offset + i-1,:,electrodos,:]
        dest.append(df_)

pd_pacientes = pd.concat(pacientes_P + pacientes_S)

In [ ]:
pacientes_b_mean = []
N_b = 0
step_b = 0
alfabeto_8 = []

# Calculo de N (cantidad de bins): Se usa n = 201 porque los epochs tienen esa cantidad de muestras
def calculo_N(df_):
    return math.ceil((df_.max() - df_.min()) / (3.5*df_.std() * 201 ** (-1/3)))

def calcular_simbolos_sensor_8_para_promediando_entre_pacientes(pacientes):
    pacientes_8 = pacientes.loc[pd.IndexSlice[:,:,8,:],:]
    val = pacientes_8.groupby(["epoch"])['valores'].mean()
    i_s = pd.DataFrame({'sensor':[8]})
    i_s['N'] = calculo_N(val)
    i_s['min'] = val.min()
    i_s['max'] = val.max()
    i_s['step'] = (i_s['max'] - i_s['min']) / i_s['N']
    return i_s

# Calcular N para hacer la transformación simbólica
def calcular_simbolos_inter_promediando_entre_pacientes(pacientes):
    pacientes_b_mean = pd.concat(pacientes_P + pacientes_S).groupby(["sensor","tiempo"]).mean()
    N_b = math.ceil(pacientes_b_mean.max() - pacientes_b_mean.min() / (pacientes_b_mean.std() * len(pacientes_b_mean) ** (-1/3)))
    step_b = (pacientes_b_mean.max() - pacientes_b_mean.min()) / N_b
    return pacientes_b_mean, N_b, step_b

def calcular_entropia_inter_electrodo_para_un_epoch(df_):
    df_['simbolos_inter'] = (df_["valores"] - pacientes_b_mean['valores'].min()) // step_b['valores']
    df_[df_['simbolos_inter'] < 0] = 0
    df_[df_['simbolos'] > N_b] = N_b
    df_['repeticiones_inter'] = df_.groupby(["simbolos_inter"]).transform('count')['valores']
    df_['probabilidad_inter'] = df_['repeticiones_inter'] / len(df_)
    p = df_["probabilidad_inter"]
    return -sum(p * np.log(p))
    
def calcular_entropia_para_un_epoch(df_):
    df_['simbolos_intra'] = (df_['valores'].values - alfabeto_8['min'].values) // alfabeto_8['step'].values
    df_[df_['simbolos_intra'] < 0] = 0
    df_[df_['simbolos_intra'] > alfabeto_8['N'].values[0]] = alfabeto_8['N'].values[0]
    df_['repeticiones_intra'] = df_.groupby(["simbolos_intra"]).transform('count')['valores']
    df_['probabilidad_intra'] = df_['repeticiones_intra'] / len(df_)
    df_ = df_.groupby("simbolos_intra").first()
    p = df_["probabilidad_intra"]
    return -sum(p * np.log(p))

# Se calcula la cantidad de Bins promediando entre todos los pacientes
alfabeto_8 = calcular_simbolos_sensor_8_para_promediando_entre_pacientes(pd_pacientes)

# Para el caso de la medida inter-electrodo se calcula la cantidad de Bins entre todos los electrodos y pacientes 
pacientes_b_mean, N_b, step_b = calcular_simbolos_inter_promediando_entre_pacientes(pd_pacientes)

In [ ]:
## Establecimos este orden para los features
def give_expected_ordered_keys():
    ordered_keys = []
    for i, forma_de_calcular_feature in enumerate(['media_','std_']):
        for  j, tipo_de_banda in enumerate(['delta', 'theta', 'alpha', 'beta', 'gamma', 'delta_norm', 'theta_norm', 'alpha_norm', 'beta_norm', 'gamma_norm', 'intra', 'inter']):
            ordered_keys.append("{}{}".format(forma_de_calcular_feature,tipo_de_banda))
    return np.array(ordered_keys)

##  Y así será al forma de los archivos pickle generados con los features (en crudo y normalizadas)
def numpy_to_pickle(aCollectionOfNumpyArray, aPickelFileName):
    print("Creando index...")
    lenght = int(len(aCollectionOfNumpyArray)/2)
    arrays_index = [
        ['P']*N_P+['S']*N_S,
        [i for j in range(2) for i in range(lenght)]
    ]

    index = pd.MultiIndex.from_arrays(arrays_index, names=["tipo", "indice_paciente"])
    arrays_columns = [
        ['media']*12+['std']*12,
        ['delta', 'theta', 'alpha', 'beta', 'gamma', 'delta_norm', 'theta_norm', 'alpha_norm', 'beta_norm', 'gamma_norm', 'intra', 'inter']*2
    ]

    index_columns = pd.MultiIndex.from_arrays(arrays_columns, names=["agrupacion_feature","feature"])

    df = pd.DataFrame(data=aCollectionOfNumpyArray, index=index, columns=index_columns)

    print(df)

    df.to_pickle("../{}.pickle".format(aPickelFileName))

In [ ]:
##Definimos el orden de los atributos antes de codear de forma rapida
##Esta clase esta solo para asegurar ese orden.
class Sample():

    def __init__(self, featuresDict):
        self.features = {}

        for k,v in featuresDict.items():
            self.features[k] = v
                
    def show_features_as_np(self):
        valores = []
        for key in give_expected_ordered_keys():
            valores.append(self.features["{}".format(key)])
        
        return np.array(valores)

In [ ]:
#Calculo la potencia de cada banda
def calcular_potencia_por_bandas_de_frecuencias(f, P):
    return {
        'delta': P[f<4].sum(),
        'theta': P[(4<=f) & (f<8)].sum(),
        'alpha': P[(8<=f) & (f<13)].sum(),
        'beta' : P[(13<=f) & (f<30)].sum(),
        'gamma': P[30<=f].sum()
    }

#  Dado un diccionario con pontencia de cada banda, 
#      calculo la potencia normalizada de cada banda.
def normalizar_banda(bandas):
    suma_del_poder_total = sum(bandas.values()) 
    bandas_norm = {}

    for key, value in bandas.items():
        bandas_norm["{}_norm".format(key)]= bandas[key]/suma_del_poder_total

    return bandas_norm

def computar_promedio_y_desvio_por_marcador_sobre_todos_los_epochs(lista_de_marcadores_por_epoch):
    features_paciente = {}

    #Junto todos los valores de un mismo marcador para usar mean y stdev de statistics
    for key, value in lista_de_marcadores_por_epoch[0].items():
        
        _values = np.array([])
        
        for dic in lista_de_marcadores_por_epoch:
            _values = np.append(_values, dic[key])

        _mean = sts.mean(_values)
        _stdev = sts.stdev(_values,xbar=_mean)

        features_paciente["media_{}".format(key)] = _mean
        features_paciente["std_{}".format(key)] = _stdev

    return Sample(features_paciente)

# De cada muestra o sample tomamos 24 features ya elegidos, computados a través del promedio y varianza
#      entre los epochs de cada paciente.
def calcular_features_de_un_paciente(paciente):
    
    marcadores_de_este_epoch = {}

    ## Junto la informacion por cada epoch
    p_ = paciente.groupby(['epoch','tiempo']).mean()
    
    lista_de_epochs = list( set(p_.loc[:,:].index.get_level_values('epoch',)) )
    lista_de_marcadores_por_epoch = []
    
    for epoch in lista_de_epochs:
        
        frecuencias = p_.loc[epoch,:]
        f, P = scipy.signal.welch(frecuencias['valores'], fs=250, nperseg=201)
        
        # Calculo todos los marcadores que necesito para un epoch
        # Estos son las bandas, las bandas normalizadas y alguna medida intra/inter electrodo
        marcadores_de_este_epoch = calcular_potencia_por_bandas_de_frecuencias(f,P)
        marcadores_de_este_epoch.update(normalizar_banda(marcadores_de_este_epoch))
        
        # Para calcular el marcador intra-electrodo solo necesito la frecuencia del sensor/electrodo 8
        frecuencias_8 = paciente.loc[pd.IndexSlice[:,epoch,8,:],:].groupby(['epoch','tiempo']).mean()
        marcadores_de_este_epoch.update({"intra":calcular_entropia_para_un_epoch(frecuencias_8)})
        
        #TODO: Cambio de Intra Inter
        marcadores_de_este_epoch.update({"inter":calcular_entropia_inter_electrodo_para_un_epoch(frecuencias)})
        lista_de_marcadores_por_epoch.append( marcadores_de_este_epoch )
    
    ## Ahora calculo los features tomando promedio y desvio sobre los marcadores de cada epoch
    sample = computar_promedio_y_desvio_por_marcador_sobre_todos_los_epochs(lista_de_marcadores_por_epoch)

    return sample

#   Usamos los mismos archivos *.hdf generados para el TP anterior.
def levantar_hdf(load_name, nth):
    paciente = load_name + "{:02d}".format(i)
    return pd.read_hdf(load_path.format(paciente))

In [ ]:
samples_pacientes = []

pacientes = list(pacientes_P + pacientes_S)
for paciente in pacientes:
    samples_pacientes.append(calcular_features_de_un_paciente(paciente))

In [ ]:
samples_pacientes[0].show_features_as_np()

In [ ]:
len(list(pacientes_P + pacientes_S))

Miremos como están quedando los datos. Un buen grafico para ver que tan esparsos son los valores de cada feature estaría bueno.

In [ ]:
#Codigo para imprimir como barplot, violinplot y swarmplot un DataFrame
def analisis_comparativo(df_banda, df_banda_estandarizado):
#    ymin = min(df_banda['Valores'])
#    ymax = max(df_banda['Valores'])
#    decimo = (ymax - ymin)/len(df_banda['Valores'])
#    ymin, ymax = ymin - decimo, ymax + decimo

    # Hay que tener en cuenta que son pocos valores

    #ax = sns.violinplot(x="Features", y="Valores", hue="Capacidad cognitiva", data=df_b,  split=True, palette="Set2", inner="stick", cut=0)
    #sns.plt.show()
    fig, (ax1,ax2) = plt.subplots(1,2, sharey=True, figsize=(12,4))
    
    #ax1
#    sns.pointplot("who", "age", data=titanic, join=False,n_boot=10, ax=ax1)

    sns.swarmplot(x="Sin Estandarizar", y="Valores", hue="Capacidad cognitiva",data=df_banda,  split=True, palette="Set2", size=4, ax=ax1)
#    ax1.set_ylim([ymin, ymax])
    
    sns.swarmplot(x="Estandarizado", y="Valores", hue="Capacidad cognitiva", data=df_banda_estandarizado,  split=True, palette="Set2", size=4, ax=ax2)
    sns.plt.show()

    #sns.barplot(x="Features", y="Valores", hue="Capacidad cognitiva", data=df_b, palette="Set2")
    #ax = sns.swarmplot(x="Features", y="Valores", hue="Capacidad cognitiva", split=True, data=df_b, palette="Set2")
    #ax.set_ylim([ymin, ymax])
    
    #handles, labels = ax.get_legend_handles_labels()
    #l = ax.legend(handles[:2], labels[:2])
    #l.set_title("Capacidad cognitiva", prop = {'size':'small'})
    #sns.plt.show() 

In [ ]:
# imprimo por feature para ver como quedaron los datos
def printear_comparacion_sin_y_con_estandarizar(nd_datos, nd_datos_stand, lista_de_keys):
    
    for indice, key in enumerate(lista_de_keys):
        valores = list([nd[indice] for nd in nd_datos])
        
        valores_estandarizados = list([nd[indice] for nd in nd_datos_stand])
        
        keys = [key]* (N_P+N_S)

        df_banda = pd.DataFrame({
            "Capacidad cognitiva": (["Disminuída"] * N_P) + (["Normal"] * N_S),
            "Sin Estandarizar": keys,
            "Valores": valores
        })
        
        df_banda_estandarizada = pd.DataFrame({
            "Capacidad cognitiva": (["Disminuída"] * N_P) + (["Normal"] * N_S),
            "Estandarizado": keys,
            "Valores": valores_estandarizados
        })
        #print ("resultado \n")
        #print (df_for_one_feature)
        analisis_comparativo(df_banda, df_banda_estandarizada)

In [ ]:
np_features_por_paciente = []
for muestra in samples_pacientes:
    np_features_por_paciente.append( muestra.show_features_as_np() )

In [ ]:
# todas las keys encadenadas
#print (len ( list(itertools.chain(*[list(give_expected_ordered_keys()) for _ in range(N_P+N_S)]))) )
# todos los valores encadenados
#print (len(list(itertools.chain(*[list(paciente) for paciente in np_features_por_paciente]))) )
# todos los headers encadenados
#print (len( (["Reducida"] * (N_P * 24)) + (["Normal"] * (N_S * 24))) )

#df_features_compress = pd.DataFrame({
#    "Capacidad cognitiva": (["Reducida"] * (N_P * 24)) + (["Normal"] * (N_S * 24)),
#    "Features": list(itertools.chain(*[list(give_expected_ordered_keys()) for _ in range(N_P+N_S)])),
#    "Valores": list(itertools.chain(*[list(paciente) for paciente in np_features_por_paciente]))
#})

In [ ]:
np_features_por_paciente

Generamos un pickle con los features en bruto.

In [ ]:
numpy_to_pickle(np.array(np_features_por_paciente, copy=True), "df_features_prueba")

Generamos también ahora un pickle con las/los features pero ahora estandarizados.

In [ ]:
a_np_copy = np.array(np_features_por_paciente, copy=True)

scaler = preprocessing.StandardScaler().fit(a_np_copy)
#print (scaler)
np_features_por_paciente_norm = scaler.transform(a_np_copy)

print ("media: ")
print(scaler.mean_)
print ("std: ")
print(scaler.var_)
print ("samples: {}".format(scaler.n_samples_seen_))

#scalerDelNorm = preprocessing.StandardScaler().fit(np_features_por_paciente_norm)
## Esto deberia devolver un vector de medias con 0s y uno de desvios con 1s, pero la media no los da.
#print ("media: ")
#print(scalerDelNorm.mean_)
#print ("std: ")
#print(scalerDelNorm.var_)

#unPasoMasScalerDelNorm = preprocessing.StandardScaler().fit(scalerDelNorm.transform(np_features_por_paciente_norm))
## Esto deberia devolver un vector de medias con 0s y uno de desvios con 1s
#print ("media: ")
#print(unPasoMasscalerDelNorm.mean_)
#print ("std: ")
#print(unPasoMasscalerDelNorm.var_)

#df_features_normalizado_compress = pd.DataFrame({
#    "Capacidad cognitiva": (["Reducida"] * (N_P * 24)) + (["Normal"] * (N_S * 24)),
#    "Features": list(itertools.chain(*[list(give_expected_ordered_keys()) for _ in range(N_P+N_S)])),
#    "Valores": list(itertools.chain(*[list(paciente) for paciente in np_features_por_paciente_norm]))
#})

In [ ]:
#print (df_features_normalizado_compress)
printear_comparacion_sin_y_con_estandarizar(np_features_por_paciente, np_features_por_paciente_norm, give_expected_ordered_keys())

In [ ]:
numpy_to_pickle(np_features_por_paciente_norm, "df_features_estandarizado")

<blockquote>
<p>Aunque visualmente no lo parezca, el desvío quedó en 1 y la medía más cerca del 0 (aunque no en 0). Algo a tener en cuenta es que estamos estandarizando por <i>feature</i>, y eso incluye tanto a los pacientes P como a los S.  
La apertura de los valores tiene sólo sentido viendo que los valores antes de estandarizar son muy pequeños.</p>
</blockquote>

In [ ]:
print(np.array([p[0] for p in np_features_por_paciente]))
print("media {}".format(np.mean(np.array([p[0] for p in np_features_por_paciente]))))
print("std {}".format(np.std(np.array([p[0] for p in np_features_por_paciente]))))

In [ ]:
print(np.array([p[0] for p in np_features_por_paciente_norm]))
print("media {}".format(np.mean(np.array([p[0] for p in np_features_por_paciente_norm]))))
print("std {}".format(np.std(np.array([p[0] for p in np_features_por_paciente_norm]))))